In [1]:
import conllu
from io import open
import pickle 
import numpy as np

In [2]:
folder = "./UD_Ukrainian-IU/"
train = "uk_iu-ud-train.conllu"
test = "uk_iu-ud-test.conllu"

In [3]:
train_data = conllu.parse_incr(open(folder+train,'r',encoding = "utf-8"))

In [4]:
c3 = {} # триграммы
c2 = {} # биграммы
c1 = {} # монограммы
cT = {} # пары слово-тег
for token in train_data:
    sentence = [["*","*"]] + [["*","*"]] + [[token[i]["lemma"], token[i]["upostag"]]
        for i in range(len(token)) ] + [["STOP","STOP"]]

    for i in range(0, len(sentence)-2):
        c3old = c3.get( (sentence[i][1], sentence[i+1][1], sentence[i+2][1]), 0)
        c3.update( { (sentence[i][1], sentence[i+1][1], sentence[i+2][1]): c3old+1 } ) 

    for i in range(0, len(sentence)-1):
        c2old = c2.get( (sentence[i][1], sentence[i+1][1]), 0)
        c2.update( { (sentence[i][1], sentence[i+1][1]): c2old+1 } )

    for i in range(0, len(sentence)):
        c1old = c1.get( (sentence[i][1]), 0)
        c1.update( { (sentence[i][1]): c1old+1 } )

    for i in range(0, len(sentence)):
        cTold = cT.get( (sentence[i][1], sentence[i][0]), 0)
        cT.update( { (sentence[i][1], sentence[i][0]): cTold+1 } )

In [5]:
print (len(c3), len(c2), len(c1),len(cT))

# Save:
with open('cs_ukr.pkl', 'wb') as f:
    pickle.dump([cT, c1, c2, c3], f)

2391 286 20 15233


In [6]:
with open('cs_ukr.pkl', 'rb') as f:
    cT, c1, c2, c3 = pickle.load(f)

In [7]:
tags = list(c1.keys())
tags.remove('*')
tags.remove('STOP')
k = len(tags) 

In [8]:

def q(s, u, v):
    eps = 1e-7
    return c3.get((u, v, s), eps) / c2.get((u, v), eps)


def e(x, s):
    eps = 1e-7
    return cT.get((s, x), eps) / c1.get((s), eps)


def K(n): # выдает возможные тэги в зависимости от положения слова в предложении (в начале предложения - *) 
    if n == -1 or n == 0:
        return ['*']
    else:
        return tags

print(K(10))

['ADP', 'NOUN', 'ADJ', 'PROPN', 'VERB', 'PUNCT', 'CCONJ', 'ADV', 'PRON', 'PART', 'DET', 'SCONJ', 'NUM', 'X', 'AUX', 'INTJ', '_', 'SYM']


Алгоритм Витерби

In [9]:
def viterbi(sent):
    Pi = {(0, '*', '*'): 1}
    bp = {}
    n = len(sent) - 2
    y = [""] * (n + 1)

    for k in range(1, n + 1):
        xk = sent[k]
       
        for u in K(k-1):
            for v in K(k):
                w = K(k-2)
                v1 = map(lambda wi:
                            Pi.get((k-1, wi, u)) *
                            q(v, wi, u) *
                            e(xk, v), w)
                v1 = list(v1)
                PiNew = max(v1)
                bpNew = w[v1.index(PiNew)]
                Pi.update({(k, u, v): PiNew})
                bp.update({(k, u, v): bpNew})

    v2 = {}
    for u in K(n-1):
        for v in K(n):
            v2.update({(n, u, v): Pi.get((n, u, v)) * q("STOP", u, v)})

    v2max = max(list(v2.values()))
    for (n, u, v), val in v2.items():
        if val == v2max:
            (y[n-1], y[n]) = (u,v)

    for k in range(n-2, 0, -1):
        y[k] = bp.get((k + 2, y[k+1], y[k+2]))

    
    return y[1:]

Прогоняем алгоритм на текстовом корпусе

In [10]:
data_test = conllu.parse_incr(open(folder+test,'r',encoding = "utf-8"))
test_tags = [] # оригинальные теги корпуса
predict_tags = [] # предсказанные алгоритмом

for token in data_test:
    sentence=["*"] + [token[i]["lemma"] for i in range(len(token)) ] + ["STOP"]
    test_tags.append([token[i]["upostag"] for i in range(len(token)) ])
    predict_tags.append(viterbi(sentence))

Считаем ошибку

In [11]:
err = 0.
for sent in range(len(test_tags)):
    err += np.mean(np.array(test_tags[sent]) != np.array(predict_tags[sent]))
print('Error = ', err/len(test_tags))

Error =  0.11879883257196387
